# Kmean to cluster skills given the skill id, the labels and the range 
Within this Notebook we perform a Kmean clustering on the labels of the skills to get a nice groupping of the skills, the model is unsupervised 
and relies of the Jaccard set distance as a distance metric to compute the centroids which the  represnet the main keywords that constitute a skill.

We use K=6, 4 accounts for multiplication, addition, substraction, division, and 2 for numberline and number_repr which gives a better clustering in terms of skills.

In [126]:
##Import data as pandas dataframe from files in data folder as sepatrate dataframes
import pandas as pd
import numpy as np
df_users = pd.read_csv('data/calcularis_small_users.csv')
df_events = pd.read_csv('data/calcularis_small_events.csv')
df_subtasks = pd.read_csv('data/calcularis_small_subtasks.csv')

In [128]:
#test if string a contains a list of strings 
def containsAny(a, ls):
    return any(i in a for i in ls)

In [129]:
games = df_events['game_name'].unique()
games = games[~pd.isnull(games)]
print(games)

#filter games for names that contain 'multiplication' case insensitive
multiplication = [game for game in games if 'multiplication' in game.lower()]
#filter games for names that contain 'division' case insensitive
division = [game for game in games if 'division' in game.lower()]
#filter games for names that contain 'addition' case insensitive
addition = [game for game in games if 'addition' in game.lower()]
#filter games for names that contain 'sustraction' case insensitive
substraction = [game for game in games if containsAny(game.lower(), ['subtraction','difference', 'sub'])]
#define number_line as set difference of games and the four categories
number_line = set(games) - set(multiplication) - set(division) - set(addition) - set(substraction)
number_line = list(number_line)

#print all categories
print('Multiplication games: ', multiplication)
print('Division games: ', division)
print('Addition games: ', addition)
print('Substraction games: ', substraction)
print('Number line games: ', number_line)

['Subitizing' 'Conversion' 'Landing' 'Comparison'
 'Estimation on Number Line' 'Distance' 'Sliderule' 'Order' 'Estimation'
 'Calculator' 'Plus-minus' 'Scale: Decomposition' 'Secret Number'
 'Distribution' 'Completion' 'Write as multiplication' 'Series'
 'Jump backwards' 'Difference' 'Scale: Subtraction' 'Shelf: Jump Ahead'
 'Calculator: Multiplication' 'Scale: Conversion' 'Shelf: given height'
 'Scale: Subtraction by Tens and Ones' 'Calculator: Division'
 'Shelf: random height' 'Calculator: Multiplication with ?'
 'Scale: Composition']
Multiplication games:  ['Write as multiplication', 'Calculator: Multiplication', 'Calculator: Multiplication with ?']
Division games:  ['Calculator: Division']
Addition games:  []
Substraction games:  ['Subitizing', 'Difference', 'Scale: Subtraction', 'Scale: Subtraction by Tens and Ones']
Number line games:  ['Jump backwards', 'Scale: Composition', 'Plus-minus', 'Sliderule', 'Shelf: random height', 'Comparison', 'Series', 'Landing', 'Order', 'Estimation

In [130]:
#show unique values of the column skill_id of the df_events dataframe
skill_ids = df_events['skill_id'].unique()
skill_ids.sort()
print(skill_ids)
print(df_events['skill_id'].unique().size)

#print count of every value of skill_id in df_events in descending order
print(df_events['skill_id'].value_counts())

#print 5 of the rows of the df_events dataframe for which skill_id is 1
print(df_events[df_events['skill_id'] == 1].head())

#is there any two different rows in df_events with the same skill_id but different game_name, print only 5 of those with the same skill_id
df_duplicated = df_events[df_events.duplicated(['skill_id', 'game_name'], keep=False)]

#print 5 rows from df_duplicated with the same game_name as first row of df_duplicated
print(df_duplicated[df_duplicated['game_name'] == df_duplicated.iloc[0]['game_name']].head())

# print unique values of game_name in df_events
print(df_events['game_name'].unique())
print(df_events['game_name'].unique().size)

#print only game_name and skill_id of 5 rows from df_events with skill_id in range 240 to 252. 
df_div = df_events[(df_events['skill_id'] >= 240) & (df_events['skill_id'] <= 252)][['game_name', 'skill_id']]

#print 5 rows of each  specific skill_id from df_div
for skill in df_div['skill_id'].unique():
    print(df_div[df_div['skill_id'] == skill].head())


# print of df_events with game_name as Calculator and unique skill_id
df_calculator = df_events[(df_events['game_name'] == 'Calculator')][['skill_id']]


# count df_calculator by skill_id
df_calculator = df_calculator['skill_id'].value_counts().reset_index()
df_calculator.columns = ['skill_id', 'count']
calculator_ids = df_calculator['skill_id'].unique().astype(int).tolist()
print(df_calculator)

[  0.   1.   3.   4.   5.   6.   7.  10.  11.  12.  13.  14.  15.  16.
  17.  18.  19.  21.  22.  23.  24.  26.  27.  28.  29.  30.  33.  35.
  36.  37.  38.  39.  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.
  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  66.  68.  69.  70.
  71.  73.  74.  75.  76.  77.  78.  79.  81.  82.  83.  84.  86.  87.
  88.  90.  91.  94.  95.  97.  98.  99. 100. 103. 104. 105. 106. 107.
 108. 109. 110. 111. 112. 114. 115. 116. 117. 118. 119. 120. 121. 122.
 123. 124. 125. 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136.
 138. 139. 140. 141. 142. 143. 144. 146. 147. 148. 149. 150. 151. 153.
 154. 155. 156. 158. 159. 160. 162. 163. 166. 167. 169. 170. 171. 172.
 175. 176. 177. 178. 179. 180. 181. 183. 184. 185. 186. 187. 188. 189.
 190. 191. 192. 193. 194. 195. 196. 197. 198. 199. 200. 201. 202. 203.
 204. 205. 206. 207. 208. 209. 210. 211. 212. 213. 215. 216. 217. 218.
 219. 221. 222. 223. 224. 225. 226. 227. 228. 229. 230. 232. 233. 234.
 235. 

In [131]:
# read and parse jso file as python dictionary
import json


# pretty print 5 items of skill_map
import pprint
pp = pprint.PrettyPrinter(indent=4)

with open('data/04_calcularis_skill_map_labels.json') as f:
    skill_map = json.load(f)['labels'] 
    pp.pprint(dict(list(skill_map.items())[0:5]))
    
    # print count of items in skill_map
    print(len(skill_map))
   
    
    # pretty print associated values to keys from skill_ids of skill_map
    for key in calculator_ids:
        print(key, " : ", skill_map[str(key)])


{   '0': '0-10 Concrete',
    '1': '0-10 Subitizing',
    '2': '0-10 Verbal',
    '3': '0-10 Concrete -> Arabic',
    '4': '0-10 Verbal -> Arabic'}
253
62  :  0-20 Subtraction TC Invisible Timelimit
59  :  0-20 Addition TC Invisible Timelimit
61  :  0-20 Subtraction TC
133  :  0-100 Subtraction 2,2 TC
30  :  0-10 Math Facts
58  :  0-20 Addition TC
71  :  0-20 Math Facts
122  :  0-100 Subtraction 2,1 TC
56  :  0-20 Subtraction 2,1
205  :  0-1000 Addition 3,2 TC, HC
27  :  0-10 Subtraction 1,1
132  :  0-100 Addition 2,2 TC
54  :  0-20 Addition 2,1
121  :  0-100 Addition 2,1 TC
117  :  0-100 Subtraction 2,1
26  :  0-10 Addition 1,1
229  :  0-1000 Subtraction 3,3 TC, HC
116  :  0-100 Addition 2,1
0  :  0-10 Concrete
186  :  0-1000 Addition 3,1 TC
189  :  0-1000 Subtraction 3,1
211  :  0-1000 Subtraction 3,2 TC, HC
227  :  0-1000 Subtraction 3,3 TC
219  :  0-1000 Addition 3,3 TC, HC
190  :  0-1000 Subtraction 3,1 TC
201  :  0-1000 Addition 3,1 HC
206  :  0-1000 Subtraction 3,1 HC
185  :  0-

In [132]:
#Put values as lower of skill_map as set and pretty print them
skill_map_values = set([value.lower() for value in skill_map.values()])
#pp.pprint(skill_map_values)


#Apply kmeans alrotithm with 5 clusters to the skill_map_values using jacard distance as distance metric on strings  
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


# Create TfidfVectorizer object with stopwords and token pattern that contains only letters
vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'[a-zA-Z]+', min_df=20)

X = vectorizer.fit_transform(skill_map_values)

true_k = 5 + 1
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1, random_state=2)
model.fit(X)

# print centroids of the clusters
print("Centroids:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()


# Fit 
skills_ls = np.array(list(skill_map_values))
in_cluster = model.predict(vectorizer.transform(skills_ls))


# predict skill_ls with model and get confidence of prediction
in_cluster = model.predict(vectorizer.transform(skills_ls))
confidence = model.transform(vectorizer.transform(skills_ls))


    


Centroids:
Cluster 0: addition support tc hc subtraction table numberline multiplication division arabic
Cluster 1: arabic numberline tc table support subtraction multiplication hc division addition
Cluster 2: division multiplication tc table support subtraction numberline hc arabic addition
Cluster 3: numberline tc table support subtraction multiplication hc division arabic addition
Cluster 4: subtraction tc support hc table numberline multiplication division arabic addition
Cluster 5: multiplication table support tc subtraction numberline hc division arabic addition


In [133]:
# create dictionnary that assocaite clusters to skills
clusters_from_idx = {4: 'Subtraction', 1: 'Number_line', 2: 'Division', 5: 'Multiplication', 0: 'Addition' ,3:'Number_repr'}

skill_sets = {}
for k in range(6):
    print('='*50)
    print('Cluster: ', clusters_from_idx[k])
    idx = np.where(in_cluster == k)[0]
    skill_sets[clusters_from_idx[k]] = skills_ls[idx]
    pp.pprint(skills_ls[idx])
    

# save skill_sets as file using pickle
import pickle
with open('skill_sets.pkl', 'wb') as f:
    pickle.dump(skill_sets, f)

Cluster:  Addition
array(['0-1000 addition 3,1 tc', '0-1000 support addition 3,1 tc',
       '0-1000 support addition 3,2',
       '0-10 support addition/subtraction structured 1',
       '0-100 addition 2,2', '0-1000 support addition 3,3 tc',
       '0-20 support addition tc', '0-20 addition',
       '0-100 support addition 2,2 tc', '0-1000 addition 3,3',
       '0-20 addition tc', '0-1000 support addition 3,3',
       '0-100 addition 2,1 tc', '0-1000 addition 3,2 tc, hc',
       '0-100 support addition 2,2', '0-1000 addition 3,3 hc',
       '0-10 addition 1,1', '0-1000 support addition 3,2 tc',
       '0-100 support addition 2,1 tc', '0-100 addition 2,2 tc',
       '0-1000 addition 3,1', '0-1000 support addition 3,1 hc',
       '0-1000 addition 3,2 hc',
       '0-10 support addition/subtraction structured 2',
       '0-1000 support addition 3,2 hc', '0-20 addition 2,1',
       '0-1000 addition 3,1 hc', '0-1000 support addition 3,3 tc, hc',
       '0-1000 addition 3,3 tc', '0-1000 sup

In [134]:
# pretty print viwew of skill_map
pp.pprint(dict(list(skill_map.items())[0:5]))

# Create bidict from skill_map using bidict library and turn values to lower case
from bidict import bidict
skill_map = bidict({int(k): v.lower() for k, v in skill_map.items()})

pp.pprint(dict(list(skill_map.items())[0:5]))


# Size of intersection between skill_map values and skill_sets values flattened
print(len(set(skill_map.values()).intersection(set([item for sublist in skill_sets.values() for item in sublist]))))


# Get key associated to value '0-10 concrete' in skill_map
print(skill_map.inv['0-10 concrete'])


# Map values of skills sets to keys of skill_map
for k, v in skill_sets.items():
    skill_sets[k] = [skill_map.inv[item] for item in v]

print(skill_sets)

# Save skill_sets as file using pickle and file name skill_sets_ids.pkl
import pickle
with open('skill_sets_ids.pkl', 'wb') as f:
    pickle.dump(skill_sets, f)

{   '0': '0-10 Concrete',
    '1': '0-10 Subitizing',
    '2': '0-10 Verbal',
    '3': '0-10 Concrete -> Arabic',
    '4': '0-10 Verbal -> Arabic'}
{   0: '0-10 concrete',
    1: '0-10 subitizing',
    2: '0-10 verbal',
    3: '0-10 concrete -> arabic',
    4: '0-10 verbal -> arabic'}
253
0
{'Addition': [186, 184, 191, 21, 125, 213, 57, 63, 130, 216, 58, 212, 121, 205, 123, 218, 26, 193, 119, 132, 185, 192, 204, 22, 194, 54, 201, 215, 217, 195, 202, 23, 183, 59, 214, 116, 219, 53, 114, 203], 'Number_line': [129, 38, 101, 3, 169, 7, 180, 4, 18, 179, 68, 35, 99, 170, 49, 39, 29, 6, 173, 171, 172, 108, 98, 37, 40, 100, 17, 8, 48, 70, 36, 226, 97, 109], 'Division': [162, 242, 249, 156, 87, 85, 241, 88, 245, 158, 247, 157, 155, 90, 246, 243, 248, 91, 83, 160, 240, 251, 163, 159, 244, 86, 84, 161, 82, 89], 'Number_repr': [11, 9, 31, 14, 15, 95, 69, 10, 46, 96, 176, 1, 111, 2, 28, 166, 0, 92, 30, 25, 33, 42, 103, 32, 231, 252, 177, 164, 41, 178, 113, 182, 52, 168, 51, 47, 181, 45, 72, 93, 137

# Future work:
In the following incomplete section (future work/extension), we attempt to use word embeddings as vectorization method to get better clustering of the skills.


In [158]:
import pandas as pd
from nltk.corpus import stopwords
import re
stop = set(stopwords.words("english"))

In [215]:
# Create Tockenizer that removes stop words and token pattern that contains only letters  and minimum document frequency of 20 using nltk library
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'[a-zA-Z]+', min_df=20)
vectorizer.fit(skill_map_values)
features_vocab = vectorizer.get_feature_names()


def preprocess(text):
    text_input = re.sub('[^a-zA-Z]+', ' ', str(text))
    output = re.sub(r'\d+', '',text_input)
    return output.lower().strip()

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

# use remove_stopwords function to remove stop words from skill_map_values
sentences = [remove_stopwords(text) for text in skill_map_values]

# use preprocess function to preprocess skill_map_values
sentences = [preprocess(text) for text in skill_map_values]

# split sentences into words
sentences = [s.split() for s in sentences]


# pretty print sentences
pp.pprint(sentences[0:5])

[   ['subtraction', 'tc'],
    ['support', 'subtraction'],
    ['larger', 'cva'],
    ['addition', 'tc'],
    ['division', 'calculate']]


In [216]:
from gensim.models import Word2Vec
model_w2v = Word2Vec(sentences, min_count=1)

In [217]:
model_w2v.wv.most_similar('addition')

[('calculate', 0.17861908674240112),
 ('sets', 0.16420435905456543),
 ('cardinal', 0.1496219038963318),
 ('arabic', 0.13352328538894653),
 ('cva', 0.09663068503141403),
 ('notation', 0.0776715874671936),
 ('table', 0.07532937824726105),
 ('multiplication', 0.06878271698951721),
 ('tables', 0.06825027614831924),
 ('relative', 0.047659650444984436)]